In [ ]:
# some of this notebook from http://bit.ly/15WsUXU
!pip install sqlalchemy

# Core

### Create an engine. For this demo we're using SQLite in memory, and echoing the SQL as it's executed. 

In [9]:
from sqlalchemy import create_engine
engine = create_engine('sqlite:///:memory:')

### Let's declare a basic table

In [10]:
from sqlalchemy import *

metadata = MetaData()

c1 = Column('model', String)
c2 = Column('registration', String)
c3 = Column('odometer', Integer)
vehicles_table = Table('vehicles', metadata, c1, c2, c3)
vehicles_table

Table('vehicles', MetaData(bind=None), Column('model', String(), table=<vehicles>), Column('registration', String(), table=<vehicles>), Column('odometer', Integer(), table=<vehicles>), schema=None)

### The table doesn't exist yet, so let's create it

In [11]:
vehicles_table.create(bind=engine)

In [13]:
ins = vehicles_table.insert()

In [14]:
print(ins)

INSERT INTO vehicles (model, registration, odometer) VALUES (:model, :registration, :odometer)


In [25]:
ins = ins.values({"model" : "Audi", "registration":"007", "odometer"})

In [26]:
print(ins)

INSERT INTO vehicles (model, registration, odometer) VALUES (:model, :registration, :odometer)


In [27]:
engine.execute(ins)

### SQLAlchemy core's main job is to generate SQL

In [39]:
query = vehicles_table.select()
print("type:",type(query))
print("query:",query)

type: <class 'sqlalchemy.sql.selectable.Select'>
query: SELECT vehicles.model, vehicles.registration, vehicles.odometer 
FROM vehicles


In [40]:
result = engine.execute(query)

In [41]:
for item in result:
    print(item)

('Alto', None, None)
('Alto', None, None)
('Audi', '007', 22)
('Audi', '007', 22)
('Honda', '006', 22)
('Wagon', '005', 22)
('Busa', '004', 22)


In [42]:
ins = vehicles_table.insert().values([{"model" : "Audi", "registration":"007", "odometer":22},
                  {"model" : "Honda", "registration":"006", "odometer":2},
                  {"model" : "Wagon", "registration":"005", "odometer":21},
                  {"model" : "Busa", "registration":"004", "odometer":29}])

In [43]:
engine.execute(ins)

### The _select_ above is a shortcut for...

In [44]:
query = select([vehicles_table])
print("type:",type(query))
print("query:",query)

type: <class 'sqlalchemy.sql.selectable.Select'>
query: SELECT vehicles.model, vehicles.registration, vehicles.odometer 
FROM vehicles


In [56]:
res = engine.execute(query)

In [57]:
print(res)

In [58]:
res.fetchall()

[('Busa', '004', 29)]

In [55]:
query = query.where(vehicles_table.c.odometer > 22)

In [ ]:
for item in res:
    print(item)

### Access the table's column metadata

In [ ]:
vehicles_table.c.odometer

### Using method chaining we can add to the query. Note how column operators are overloaded to produce SQL

In [ ]:
query = query.where(vehicles_table.c.odometer < 10000)


print(query)

In [60]:
vehicles_table.c.model

Column('model', String(), table=<vehicles>)

In [61]:
import pandas as pd

In [62]:
print(query)

SELECT vehicles.model, vehicles.registration, vehicles.odometer 
FROM vehicles 
WHERE vehicles.odometer > :odometer_1 AND vehicles.odometer > :odometer_2


In [64]:
pd.read_sql_query("select * from vehicles", engine)

,model,registration,odometer
0,Alto,None,NaN
1,Alto,None,NaN
2,Audi,007,22.0
3,Audi,007,22.0
4,Honda,006,22.0
5,Wagon,005,22.0
6,Busa,004,22.0
7,Audi,007,22.0
8,Honda,006,2.0
9,Wagon,005,21.0


### Using the engine created earlier, let's now run our query

In [ ]:
results = engine.execute(query)
for row in results:
    print(row,end=" ")

### No results, let's insert some rows

In [ ]:
values = [
    { 'model': 'Ford Festiva', 'registration': 'HAX00R', 'odometer': 3141 },
    { 'model': 'Lotus Elise', 'registration': 'DELEG8', 'odometer': 31415 },
]
rows = engine.execute(vehicles_table.insert(), list(values)).rowcount
print(rows, "rows inserted")

### Try our query again

In [ ]:
results = engine.execute(query)
for row in results:
    print(row,end=" ")

### Trying something more elaborate, let's see the SQL for adding up the odometer readings by model 

In [ ]:
print(select([vehicles_table.c.model, 
              func.sum(vehicles_table.c.odometer).label('total_km')
             ]).group_by(vehicles_table.c.model))

### There's much much more we can do. Literal SQL, functions, joins, aliases, unions, ...

# ORM

In [65]:
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import relationship
Base = declarative_base()

class Person(Base):
    __tablename__ = 'people' # Choose your own table name!
    id = Column(Integer, primary_key=True)
    first_name = Column(String)
    last_name = Column(String)
    email = Column(String)
    birthday = Column(DateTime)
    appointments = relationship("Appointment", backref="person")

    def __init__(self, firstname, lastname, email):
        """ Constructor is optional """
        self.first_name = firstname
        self.last_name = lastname
        self.email = email

class Appointment(Base):
    __tablename__ = 'appointments'
    id = Column(Integer, primary_key=True)
    person_id = Column(Integer, ForeignKey('people.id')) # <-- Table name
    meeting_at = Column(DateTime)
    notes = Column(String)

In [ ]:
print("Underlying table object:\n", repr(Person.__table__))
print("*"*30)
print("Mapper that's taking care of things:\n", repr(Person.__mapper__))
print("*"*30)
print("What does the declarative base know?\n", repr(Base.metadata.tables))

### Create the table!

In [ ]:
Base.metadata.create_all(engine) 

### Now let's create a session

In [ ]:
from sqlalchemy.orm import sessionmaker
Session = sessionmaker(bind=engine)
session = Session()

p = Person('newfirst', 'newlast', 'new@example.com');
session.add(p)

### The query below will cause the above addition to be automatically flushed.

In [ ]:
for p in session.query(Person):
    print(p.first_name)

### The ORM uses the underlying core to generate SQL

In [ ]:
print(session.query(Person).filter_by(id=1))

### Session queries can be built up to retrieve specific elements

In [ ]:
for row in session.query(Person, Person.first_name, Person.email.label('address')).filter_by(id=1):
    print("Person object:", row.Person)
    print("Selected attributes:", row.first_name, row.address)

### The session keeps track of objects and their state, so the exact same object is returned even via another query.

In [ ]:
queried_person = session.query(Person).filter_by(first_name='newfirst').first()

p is queried_person

### Queries are generative

In [ ]:
query = session.query(Person).filter(~Person.first_name.in_(['ed', 'wendy', 'jack']))
print(query)

### Additional filter criteria is added with an AND operator

In [ ]:
from datetime import datetime, timedelta
query = query.filter(Person.birthday < datetime.now() - timedelta(days=1))
print(query)

# Relationships

### Let's create an appointment related to our person

In [ ]:
appointment = Appointment(person=session.query(Person).first(),
                          notes="Appointment date TBC")
session.add(appointment)

### Now we can query from either end of the relationship

In [ ]:
for a in session.query(Appointment):
    print(a.person)

In [ ]:
for p in session.query(Person):
    print(p.appointments)